In [245]:
#Python 3.11.2
#Import packages
import pandas as pd
import numpy as np
import pylab as pl
import random
import matplotlib.pyplot as plt
from collections import defaultdict
import json # for pretty printing
import geopandas as gpd

We have three different data sources. 

1. The data collected by Bodil corresponds to the plague period.
2. The information from the TABVERK database includes the population size for parishes in the posterior years of the plague.
3. The geographical information (polygons) for some parishes. This information doesn't correspond to the plague period.

Our goal is to create a unique database for our project: Plague spread across Scania, Sweden, from 1710 to 1715.

First we read the different data sources (.xlsx, .csv, and .shp files)

In [288]:
allParishesScania_path = '/Users/polislizarralde/PythonMathematicalModeling/docs/PlagueProject/data/allParishesScania.xlsx'
allParishesScania = pd.read_excel(allParishesScania_path)

Transforming the lowercase to uppercase and checking the type

In [289]:
allParishesScania = allParishesScania.apply(lambda x: x.astype(str).str.upper())
allParishesScania.head()
type(allParishesScania)

pandas.core.frame.DataFrame

Defining a function for extracting the names of the parishes in the data frame

In [290]:
def get_Names(data: pd.DataFrame, heading:str) -> list:
    return data[heading].tolist()   

len(get_Names(allParishesScania, 'Parish Name')) 

398

Filtering the data frame by region and then get the names of the parishes:

In [291]:
southeastParishes = allParishesScania.loc[allParishesScania['Region'] == 'SOUTHEAST']
southeastParishes_names = get_Names(southeastParishes, 'Parish Name')

In [292]:
southeastParishes_names[1]

'BRÖSARP'

Reading the census file:

In [246]:
census_path = '/Users/polislizarralde/Desktop/CensusScania/FILE01_FALD.csv'
censusSweden = pd.read_csv(census_path, sep=';')
censusSweden.shape

(102360, 50)

Checking the memory usage

In [247]:
censusSweden.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102360 entries, 0 to 102359
Data columns (total 50 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   LANGEN       102360 non-null  int64  
 1   LANGENNMN    102360 non-null  object 
 2   GEOID        102360 non-null  int64  
 3   GEOIDNMN     102360 non-null  object 
 4   GEOIDTYP     102360 non-null  int64  
 5   AR           102360 non-null  int64  
 6   FORMNR       102360 non-null  int64  
 7   KON          102360 non-null  int64  
 8   FORMID       102360 non-null  int64  
 9   ALD00        102360 non-null  int64  
 10  ALD01        102360 non-null  int64  
 11  ALD03        102360 non-null  int64  
 12  ALD03_1      102360 non-null  int64  
 13  ALD05        102360 non-null  int64  
 14  ALD05_2      102360 non-null  int64  
 15  ALD10        102360 non-null  int64  
 16  ALD15        102360 non-null  int64  
 17  ALD15_1      102360 non-null  int64  
 18  ALD15_2      102360 non-

Checking the names of all columns in the data

In [248]:
censusSweden.columns

Index(['LANGEN', 'LANGENNMN', 'GEOID', 'GEOIDNMN', 'GEOIDTYP', 'AR', 'FORMNR',
       'KON', 'FORMID', 'ALD00', 'ALD01', 'ALD03', 'ALD03_1', 'ALD05',
       'ALD05_2', 'ALD10', 'ALD15', 'ALD15_1', 'ALD15_2', 'ALD20', 'ALD25',
       'ALD30', 'ALD35', 'ALD40', 'ALD45', 'ALD50', 'ALD55', 'ALD60',
       'ALD60_1', 'ALD60_2', 'ALD65', 'ALD70', 'ALD75', 'ALD80', 'ALD85',
       'ALD90', 'ALD90_N', 'ALD95', 'ALD101_N', 'ALD100', 'ALD101', 'ALD102',
       'ALD103', 'ALD104', 'ALD105_N', 'BEF_SUM', 'BEF_TOT', 'BEF_GENSUM',
       'BEF_GENTOT', 'Unnamed: 49'],
      dtype='object')

Calling the data only with specific columns

In [249]:
censusSweden = pd.read_csv(census_path, sep=';', usecols=[
                           'LANGEN', 'LANGENNMN', 'GEOID', 'GEOIDNMN', 'GEOIDTYP', 'AR', 'FORMNR', 'KON', 'FORMID', 'BEF_SUM', 'BEF_TOT', 'BEF_GENSUM', 'BEF_GENTOT'])


Processing the census data such that corresponds only to Scania

In [250]:
censusScania = censusSweden.loc[((censusSweden['LANGENNMN'] == 'KRISTIANSTADS LÄN') | (
    censusSweden['LANGENNMN'] == 'MALMÖHUS LÄN')) & (censusSweden['KON'] == 1)]
censusScania.shape

(8748, 13)

Process the data from Scania only to keep the first population size registered for each parish.
This was done following two approaches: the first one by grouping the data by parish name and then selecting the minimum year. The second approach is exhaustively exploring the given DataFrame and keeping the required information in a dictionary. The information required in our case corresponds to the position associated with each parish name and the minimum year, according to the original DataFrame. 

1. First approach:

In [251]:
#Group a Pandas DataFrame by a column
parish_grp = censusScania.groupby(['GEOIDNMN'])
#Get a group by an element of the column
example = parish_grp.get_group('SANKT MARIA (YSTAD SF)')
#Select the row with minimum value in Pandas dataframe
example[example.AR == example.AR.min()]

#Get the unique values of a column
parish_grp['GEOIDNMN'].unique()
#Get the unique values of a column as a list
parish_grp_name = parish_grp['GEOIDNMN'].unique().tolist()
#Get the unique values of a column as a list of strings
parish_names = [parish_name[0] for parish_name in parish_grp_name]


In [271]:
parish_grp = censusScania.groupby(['GEOIDNMN'])
parish_grp_name = parish_grp['GEOIDNMN'].unique().tolist()
parish_names = [parish_name[0] for parish_name in parish_grp_name]

popSizeScania = pd.DataFrame() 

for i in range(len(parish_grp)):
    grp_i = parish_grp.get_group(parish_names[i])
    grp_i[grp_i.AR == grp_i.AR.min()]
    popSizeScania = pd.concat([popSizeScania, (grp_i[grp_i.AR == grp_i.AR.min()])], axis=0)

print(popSizeScania.shape)  

(495, 13)


In [272]:
popSizeScania

,LANGEN,LANGENNMN,GEOID,GEOIDNMN,GEOIDTYP,AR,FORMNR,KON,FORMID,BEF_SUM,BEF_TOT,BEF_GENSUM,BEF_GENTOT
56846,12,MALMÖHUS LÄN,72430,ALLERUM,0,1783,200,1,140161,754,1498,746,1497
48610,12,MALMÖHUS LÄN,69119,ALSEDA PASTORAT,1,1795,200,1,119630,134,252,134,252
51476,12,MALMÖHUS LÄN,70720,ANDERSLÖV,0,1749,100,1,126836,300,629,300,629
41074,11,KRISTIANSTADS LÄN,72790,ANDRARUM,0,1749,100,1,101044,695,1454,695,1454
56026,12,MALMÖHUS LÄN,72130,ANNELÖV,0,1775,200,1,138203,195,394,195,393
...,...,...,...,...,...,...,...,...,...,...,...,...,...
54826,12,MALMÖHUS LÄN,71720,ÖSTRABY,0,1749,100,1,135240,355,781,355,781
54480,12,MALMÖHUS LÄN,71630,ÖVED,0,1749,100,1,134353,223,456,223,456
42098,11,KRISTIANSTADS LÄN,73100,ÖVRABY,0,1775,200,1,103666,142,299,142,299
42110,11,KRISTIANSTADS LÄN,73101,"ÖVRABY, DEL (KRISTIANSTAD)",9,1805,310,1,103702,150,300,150,300


2. Second approach

In [270]:
aux_dict = {}
popSizeScania = pd.DataFrame() 

for i in range(len(censusScania)):
    name_i = censusScania['GEOIDNMN'].iloc[i]
    ar_i = censusScania['AR'].iloc[i]
    if name_i in aux_dict:
        if ar_i < aux_dict[name_i]['min']:
            aux_dict[name_i] = {'min': ar_i, 'position': i}
    else:
        aux_dict[name_i] = {'min': ar_i, 'position': i}
final_positions = [value['position'] for key, value in aux_dict.items()]
popSizeScania = censusScania.iloc[final_positions]   
print(popSizeScania.shape)            
    

(495, 13)


In [293]:
southeastParishes_names

['ANDRARUM',
 'BRÖSARP',
 'ELJARÖD',
 'FÅGELTOFTA',
 'RAVLUNDA',
 'RÖRUM',
 'SANKT OLOF',
 'SÖDRA MELLBY',
 'VITABY',
 'BALDRINGE',
 'BJÄRESJÖ',
 'BORRIE',
 'BROMMA',
 'HEDESKOGA',
 'HÖGESTA',
 'STORA HERRESTA',
 'SÖVESTAD',
 'TRANÅS',
 'YSTAD SANKT PETRI',
 'YSTAD SANKT MARIA',
 'ÖJA',
 'BENESTAD',
 'BOLLERUP',
 'GLEMMINGE',
 'HAMMENHÖG',
 'HANNAS',
 'HÖRUP',
 'INGELSTORP',
 'KVERRESTAD',
 'LÖDERUP',
 'ONSLUNDA',
 'SMEDSTORP',
 'SPJUTSTORP',
 'STORA KÖPINGE',
 'TOSTERUP',
 'TRYDE',
 'ULLSTORP',
 'VALLEBERGA',
 'ÖSTRA HERRESTA',
 'ÖSTRA HOBY',
 'ÖSTRA INGELSTAD',
 'ÖVRABY',
 'BOLSHÖG',
 'BORRBY',
 'GLADSAX',
 'JERRESTAD',
 'SIMRIS',
 'SIMRISHAMN',
 'STIBY',
 'VALLBY',
 'ÖSTRA NÖBBELÖV',
 'ÖSTRA TOMMARP',
 'ÖSTRA VEMMERLÖV',
 'BALKÅKRA',
 'KATSLÖSA',
 'SKÅRBY',
 'SKÖRUP',
 'SNÅRESTAD',
 'VILLIE',
 'VÄSTRA NÖBBELÖV']

In [294]:
def check_name(data: pd.DataFrame, column: pd.Series, name: str):
    filt_name = column.str.contains(name, na=False)
    return data.loc[filt_name]

#check_name(popSizeScania, popSizeScania['GEOIDNMN'], southeastParishes_names)


In [295]:
southeastParishesNoPop = []

for name in southeastParishes_names:
    filter_data = check_name(popSizeScania, popSizeScania['GEOIDNMN'], name)
    if len(filter_data) == 0 :
        southeastParishesNoPop = southeastParishesNoPop + [name]
    else:
        continue
    print(southeastParishesNoPop)
    

['SANKT OLOF']
['SANKT OLOF', 'YSTAD SANKT PETRI']
['SANKT OLOF', 'YSTAD SANKT PETRI', 'YSTAD SANKT MARIA']
['SANKT OLOF', 'YSTAD SANKT PETRI', 'YSTAD SANKT MARIA', 'JERRESTAD']
['SANKT OLOF', 'YSTAD SANKT PETRI', 'YSTAD SANKT MARIA', 'JERRESTAD', 'SKÖRUP']


In [287]:
check_name(popSizeScania, popSizeScania['GEOIDNMN'], 'YSTAD')
check_name(popSizeScania, popSizeScania['GEOIDNMN'], 'OLOFS')


,LANGEN,LANGENNMN,GEOID,GEOIDNMN,GEOIDTYP,AR,FORMNR,KON,FORMID,BEF_SUM,BEF_TOT,BEF_GENSUM,BEF_GENTOT


In [213]:
type(censusScania['GEOIDNMN'])

pandas.core.series.Series

In [220]:
filt2 = popSizeScania['GEOIDNMN'].str.contains('NORRA RÖRUM', na=False)
popSizeScania.loc[filt2]

,LANGEN,LANGENNMN,GEOID,GEOIDNMN,GEOIDTYP,AR,FORMNR,KON,FORMID,BEF_SUM,BEF_TOT,BEF_GENSUM,BEF_GENTOT
40902,11,KRISTIANSTADS LÄN,71342,"NORRA RÖRUM, DEL (KRISTIANSTAD)",9,1855,610,1,100573,169,346,169,346
53446,12,MALMÖHUS LÄN,71340,NORRA RÖRUM,0,1785,200,1,131832,376,783,376,783
53440,12,MALMÖHUS LÄN,71349,NORRA RÖRUM PASTORAT,1,1773,100,1,131817,587,1228,587,1228
53472,12,MALMÖHUS LÄN,71341,"NORRA RÖRUM, DEL (MALMÖHUS)",9,1855,610,1,131914,475,978,475,978


In [218]:
print(popSizeScania[['GEOIDNMN','AR']].iloc[2])

GEOIDNMN    NORRA RÖRUM, DEL (KRISTIANSTAD)
AR                                     1855
Name: 40902, dtype: object
